## 腾讯社交广告算法大赛
## 模型训练-Random Forest

In [1]:
from sklearn.tree import DecisionTreeClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

### 准备数据

In [2]:
dpath = "./data/"

In [3]:
train_merge = pd.read_csv(dpath+"train_merge.csv")
train_merge.head()

,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator,clickTime_day,clickTime_hour,...,adID,camgaignID,advertiserID,appID,appPlatform,appCategory,app_categories_first_class,app_categories_second_class,sitesetID,positionType
0,0,170000,NaN,3089,2798058,293,1,1,17,0,...,1321,83,10,434,1,108,1,8,0,3
1,0,180028,NaN,3089,1683269,293,2,3,18,0,...,1321,83,10,434,1,108,1,8,0,3
2,0,191045,NaN,3089,240899,293,1,2,19,1,...,1321,83,10,434,1,108,1,8,0,3
3,0,182300,NaN,2230,2177495,293,2,1,18,3,...,2841,83,10,434,1,108,1,8,0,3
4,0,200113,NaN,2230,417301,293,2,2,20,0,...,2841,83,10,434,1,108,1,8,0,3


In [4]:
train_merge.columns

Index(['label', 'clickTime', 'conversionTime', 'creativeID', 'userID',
       'positionID', 'connectionType', 'telecomsOperator', 'clickTime_day',
       'clickTime_hour', 'age', 'gender', 'education', 'marriageStatus',
       'haveBaby', 'hometown', 'residence', 'age_process', 'hometown_province',
       'hometown_city', 'residence_province', 'residence_city', 'adID',
       'camgaignID', 'advertiserID', 'appID', 'appPlatform', 'appCategory',
       'app_categories_first_class', 'app_categories_second_class',
       'sitesetID', 'positionType'],
      dtype='object')

In [5]:
train_merge["label"].value_counts()

0    3656266
1      93262
Name: label, dtype: int64

In [6]:
X_train = train_merge.loc[:,['creativeID', 'userID',
       'positionID', 'connectionType', 'telecomsOperator', 'clickTime_day',
       'clickTime_hour', 'age', 'gender', 'education', 'marriageStatus',
       'haveBaby', 'residence', 'age_process', 'hometown_province',
       'hometown_city', 'residence_province', 'residence_city', 'adID',
       'camgaignID', 'advertiserID', 'appID', 'appPlatform',
       'app_categories_first_class', 'app_categories_second_class',
       'sitesetID', 'positionType']]
X_train = np.array(X_train.values,dtype='int32')
y_train = train_merge.iloc[:,0].values

In [7]:
X_train.shape, y_train.shape

((3749528, 27), (3749528,))

### 模型训练

In [9]:
%%time
from sklearn.ensemble import RandomForestClassifier
params =  dict(n_estimators=200, max_depth=8,\
               min_samples_split=100, min_samples_leaf=10,\
               random_state=6, criterion='entropy',\
               max_features=8, verbose = 1, n_jobs=4, bootstrap=False)
rf = RandomForestClassifier(**params)
rf.fit(X_train,y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 12.3min


Wall time: 12min 45s


[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed: 12.7min finished


In [10]:
y_proba = rf.predict_proba(X_train)
auc = roc_auc_score(y_train,y_proba[:,1])
print(auc)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   23.4s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:   24.5s finished


0.7562835299162292


In [ ]:
pickle.dump(rf,open("./model/rf.pkl","wb"))
pickle.dump(y_proba,open("./result/rf_pred.pkl","wb"))